In [1]:
import sys
quic_path = '/home/quic/QUIC-Projects'
if not quic_path in sys.path:
    sys.path.append(quic_path)

In [2]:
import pennylane as qml
import pennylane.numpy as np
from matplotlib import pyplot as plt

In [3]:
# for SVM
from classifiers.convex.svm import BinarySVM
from classifiers.kernel import Kernel
from classifiers.datasets.dataloader import Example_4x2
from classifiers.datasets import IrisDataset
# for QASVM
from classifiers.quantum.qasvm import QASVM, NormQSVM
from classifiers.quantum.ansatz import Circuit9, sample_circuit
from classifiers.optimizer import tSPSA
from classifiers.callback import CostParamStorage
from qiskit.utils import QuantumInstance
from qiskit.utils.mitigation import CompleteMeasFitter
from qiskit.providers.aer import AerSimulator, StatevectorSimulator

In [4]:
ds = IrisDataset(feature_range=(-np.pi, np.pi), true_hot=0)
X, y = ds.sample(2**6, return_X_y=True)

In [5]:
_feature_map = Circuit9(X.shape[1], reps=1)
feature_map = _feature_map.copy()
for _ in range(2 - 1):
    feature_map.compose(_feature_map, inplace=True)

var_form = sample_circuit(15)(num_qubits=6, reps=5)
quantum_instance = QuantumInstance(backend=StatevectorSimulator())
qasvm = NormQSVM(
        X, y, lamda=1,
        quantum_instance=quantum_instance,
        var_form=var_form,
        feature_map=feature_map
    )
storage = CostParamStorage()
optimizer = tSPSA(maxiter=2**5, blocking=True, last_avg=16, callback=storage)

In [19]:
from qiskit import transpile
MyQC = qml.from_qiskit(transpile(var_form, basis_gates=['rx', 'ry', 'rz', 'cx', 'cz']))
MyQC

<function pennylane_qiskit.converter.load.<locals>._function(params: dict = None, wires: list = None)>

In [20]:
dev = qml.device('default.qubit', wires=var_form.num_qubits, shots=1000)

@qml.qnode(dev)
def circuit():
    MyQC(wires=range(var_form.num_qubits))
    return qml.state()

print(qml.draw(circuit)())

ValueError: The parameter θ[0] was not bound correctly.

In [16]:
circuit()

/home/quic/anaconda3/envs/qiskit/lib/python3.9/site-packages/pennylane_qiskit/converter.py:234: UserWarning: pennylane_qiskit.converter: The Gate instruction is not supported by PennyLane, and has not been added to the template.
  warnings.warn(
/home/quic/anaconda3/envs/qiskit/lib/python3.9/site-packages/pennylane_qiskit/converter.py:234: UserWarning: pennylane_qiskit.converter: The UCRYGate instruction is not supported by PennyLane, and has not been added to the template.
  warnings.warn(
/home/quic/anaconda3/envs/qiskit/lib/python3.9/site-packages/pennylane_qiskit/converter.py:234: UserWarning: pennylane_qiskit.converter: The UCRXGate instruction is not supported by PennyLane, and has not been added to the template.
  warnings.warn(


tensor(1., requires_grad=True)

In [ ]:
class NeoQASVM(object):
    def __init__(self, X, y, feature_map, var_form, lamda=1):
        self.data = X
        self.label = y
        self.feature_map = feature_map
        self.var_form = var_form
        self.lamda = lamda

    def 